# Julia is as fast as compiled C, Fortran

## Performance benchmark, Kuramoto-Sivashinksy equation

The Kuramoto-Sivashinsky (KS) equation is a nonlinear time-evolving partial differential equation (PDE) on a 1d spatial domain.

\begin{equation*}
u_t = - u_{xx} - u_{xxxx} - u u_x
\end{equation*}

where $x$ is space, $t$ is time, and subscripts indicate differentiation. We assume a spatial domain $x \in [0, L_x]$ with periodic boundary conditions and initial condition $u(x,0) = u_0(x)$. 

A simulation for $u_0 = \cos(x) + 0.1 \sin x/8  + 0.01 \cos x/32$ and $L_x = 64\pi$.
![Kuramoto-Sivashinsky u(x,t)](figs/ksdynamics.svg)

## Mathematics of benchmark algorithm, KS-CNAB2

Start from the Kuramoto-Sivashinsky equation $[0,L]$ with periodic boundary conditions

\begin{equation*}
u_t = - u_{xx} - u_{xxxx} - u u_{x}
\end{equation*}

on the domain $[0,L_x]$ with periodic boundary conditions and initial condition $u(x,0) = u_0(x)$. We will use a finite Fourier expansion to discretize space and finite-differencing to discretize time, specifically the 2nd-order rank-Nicolson/Adams-Bashforth (CNAB2) timestepping formula. CNAB2 is low-order but straightforward to describe and easy to implement for this simple benchmark.

Write the KS equation as 

\begin{equation*}
u_t = Lu + N(u)
\end{equation*}

where $Lu = - u_{xx} - u_{xxxx}$ is the linear terms and $N(u) = -u u_{x}$ is the nonlinear term. In practice we'll calculate the $N(u)$ in the equivalent form $N(u) = - 1/2 \, d/dx \, u^2$. 

Discretize time by letting $u^n(x) = u(x, n\Delta t)$ for some small $\Delta t$. The CNAB2 timestepping forumale approximates  $u_t = Lu + N(u)$ at time $t = (n+1/2) \, dt$ as 

\begin{equation*}
\frac{u^{n+1} - u^n}{\Delta t} = L\left(u^{n+1} + u^n\right) + \frac{3}{2} N(u^n) - \frac{1}{2} N(u^{n-1})
\end{equation*}


Put the unknown future $u^{n+1}$'s on the left-hand side of the equation and the present $u^{n}$ and past $u^{n+1}$ on the right.

\begin{equation*}
\left(I  - \frac{\Delta t}{2} L \right) u^{n+1} = \left(I  + \frac{\Delta t}{2}L \right) u^{n} + \frac{3 \Delta t}{2} N(u^n) - \frac{\Delta t}{2} N(u^{n-1})
\end{equation*}
Note that the linear operator $L$ applies to the unknown $u^{n+1}$ on the LHS, but that the nonlinear operator $N$ applies only to the knowns $u^n$ and $u^{n-1}$ on the RHS. This is an *implicit* treatment of the linear terms, which keeps the algorithm stable for large time steps, and an *explicit* treament of the nonlinear term, which makes the timestepping equation linear in the unknown $u^{n+1}$.

Now we discretize space with a finite Fourier expansion, so that $u$ now represents a vector of Fourier coefficients and $L$ turns into matrix (and a diagonal matrix, since Fourier modes are eigenfunctions of the linear operator). Let matrix $A = (I  - \Delta t/2 \; L)$, matrix $B =  (I  + \Delta t/2 \; L)$, and let vector $N^n$ be the Fourier transform of a collocation calculation of $N(u^n)$. That is, $N^n$ is the Fourier transform of $- u u_x = - 1/2 \, d/dx \, u^2$ calculated at $N_x$ uniformly spaced gridpoints on the domain $[0, L_x]$. 

With the spatial discretization, then the CNAB2 timestepping formula becomes 

\begin{equation*}
A \, u^{n+1} = B \, u^n + \frac{3 \Delta t}{2} N^n -  \frac{\Delta t}{2}N^{n-1}
\end{equation*}

This is a simple $Ax=b$ linear algebra problem whose iteration approximates the time-evolution of the Kuramoto-Sivashinksy PDE. With the Fourier discretization, $A$ and $B$ are diagonal. 

## KS-CNAB2 codes

We implemented the KS-CNAB2 numerical integration algorithm in six languages: Python, Matlab, Julia, C, C++, and Fortran-90. All languages used the same FFTW library for Fourier transforms. 

  * [ksbenchmark.py](ks-codes/ksbenchmark.py), Python
  * [ksbenchmark.m](ks-codes/ksbenchmark.m), Matlab
  * [ksbenchmark.jl](ks-codes/ksbenchmark.jl), Julia 
  * [ksbenchmark.c](ks-codes/ksbenchmark.c), C
  * [ksbenchmark.cpp](ks-codes/ksbenchmark.cpp), C++ 
  * [ksbenchmark.f90](ks-codes/ksbenchmark.f90), Fortran


### Matlab

The Matlab implementation is about 30 lines of code, with plotting and excluding whitespace and comments.

```
function t,U = ksintegrate(u, Lx, dt, Nt, nsave)
  Nx = length(u);                % number of gridpoints
  kx = [0:Nx/2-1 0 -Nx/2+1:-1];  % integer wavenumbers: exp(2*pi*kx*x/L)
  alpha = 2*pi*kx/Lx;            % real wavenumbers:    exp(alpha*x)
  D = i*alpha;                   % D = d/dx operator in Fourier space
  L = alpha.^2 - alpha.^4;       % linear operator -D^2 - D^3 in Fourier space
  G = -0.5*D;                    % -1/2 D operator in Fourier space

  Nsave = round(Nt/nsave) + 1    % number of saved time steps
  t = (0:Nsave)*(dt*nsave)       % t timesteps
  U = zeros(Nsave, Nx)           % matrix of u(xⱼ, tᵢ) values
  U(1,:) = u                     % assign initial condition to U
  s = 2                          % counter for saved data
   
  % some convenience variables
  dt2  = dt/2;
  dt32 = 3*dt/2;
  A_inv = (ones(1,Nx) - dt2*L).^(-1);
  B     =  ones(1,Nx) + dt2*L;

  % compute nonlinear term Nn = -u u_x
  Nn  = G.*fft(u.*u);    % Nn = -1/2 d/dx u^2 = -u u_x
  Nn1 = Nn;              % Nn1 = Nn at first time step
  u = fft(u);            % transform u physical -> spectral
  
  % timestepping loop  
  for n = 1:Nt
      
    Nn1 = Nn;                       % shift nonlinear term in time
    Nn  = G.*fft(real(ifft(u)).^2); % compute Nn = N(u) = -1/2 d/dx u^2 = -u u_x

    % time-stepping eqn: Matlab generates six temp vectors to evaluate
    u = A_inv .* (B .* u + dt32*Nn - dt2*Nn1);

    if mod(n, nsave) == 0
       U(s,:) = real(ifft(u));
          s = s+1;            
    end

end
```

### Julia 

The Julia implementation is 38 lines of code, with plotting and excluding whitespace and comments. The 8 extra lines of code over Matlab are for in-place Fourier transforms.

In [1]:
function ksintegrate(u0, Lx, dt, Nt, nsave);
    u = (1+0im)*u0                      # force u to be complex
    Nx = length(u)                      # number of gridpoints
    kx = vcat(0:Nx/2-1, 0:0, -Nx/2+1:-1)# integer wavenumbers: exp(2*pi*kx*x/L)
    alpha = 2*pi*kx/Lx                  # real wavenumbers:    exp(alpha*x)
    D = 1im*alpha                       # spectral D = d/dx operator 
    L = alpha.^2 - alpha.^4             # spectral L = -D^2 - D^4 operator
    G = -0.5*D                          # spectral -1/2 D operator
    
    Nsave = div(Nt, nsave)+1            # number of saved time steps, including t=0
    t = (0:Nsave)*(dt*nsave)            # t timesteps
    U = zeros(Nsave, Nx)                # matrix of u(xⱼ, tᵢ) values
    U[1,:] = u                          # assign initial condition to U
    s = 2                               # counter for saved data
    
    # convenience variables
    dt2  = dt/2
    dt32 = 3*dt/2
    A_inv = (ones(Nx) - dt2*L).^(-1)
    B     =  ones(Nx) + dt2*L
    
    # compute in-place FFTW plans
    FFT! = plan_fft!(u, flags=FFTW.ESTIMATE)
    IFFT! = plan_ifft!(u, flags=FFTW.ESTIMATE)

    # compute nonlinear term Nn == -u u_x 
    Nn  = G.*fft(u.^2);    # Nn == -1/2 d/dx (u^2) = -u u_x
    Nn1 = copy(Nn);        # Nn1 = Nn at first time step
    FFT!*u;
    
    # timestepping loop
    for n = 1:Nt

        Nn1 .= Nn       # shift nonlinear term in time
        Nn .= u         # put u into Nn in prep for comp of nonlinear term
        
        IFFT!*Nn;       # transform Nn to gridpt values, in place
        Nn .= Nn.*Nn;   # collocation calculation of u^2
        FFT!*Nn;        # transform Nn back to spectral coeffs, in place

        Nn .= G.*Nn;    # compute Nn = N(u) = -1/2 d/dx u^2 = -u u_x

        # loop fusion! Julia translates this line into a single for-loop on
        #   u[i] = A_inv[i] * (B[i]*u[i] + dt32*Nn[i] - dt2*Nn1[i]; 
        # no temporary vectors! 

        u .= A_inv .* (B .* u .+ dt32.*Nn .- dt2.*Nn1); 
        
        if mod(n, nsave) == 0
            U[s,:] = real(ifft(u))
            s += 1            
        end
    end
   
    t,U
end

ksintegrate (generic function with 1 method)

## Execute the Julia code

In [2]:
using Plots; gr() # load the plotting module

Plots.GRBackend()

In [5]:
# set parameters
Lx = 64*pi
Nx = 1024
dt = 1/16
nsave = 8
Nt = 3200

# set initial condition and run simulation
x = Lx*(0:Nx-1)/Nx
u = cos.(x) + 0.1*sin.(x/8) + 0.01*cos.((2*pi/Lx)*x);
t,U = ksintegrate(u, Lx, dt, Nt, nsave)

# plot results
heatmap(x,t,U, xlim=(x[1], x[end]), ylim=(t[1], t[end]), xlabel="x", ylabel="t", 
    title="u(x,t)", fillcolor=:bluesreds)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 u(x,t) 
 
 
 x 
 
 
 t 
 
 
 
 
 
 
 
 
 
 
<image width="18" height="329" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAFJCAYAAACW8H3XAAABtUlEQVR4nO2awQ3DMAwD5SDbd+ek
IzCPexwEagBCZnSVLPf8Zt4B4iJEZmbuOYcROheT1D0XkxHn0TnY0SChA3l0D3Q00GzsaJjZECIg
tFwdyaDFWFOaXWiTkLCydfQX2hiLocUyUppdaJOQDlrjwM7VUaFNQjposYxIs33Q2jza22mNZtt+
/IXQGgtS9tVakDmMLZvqtLprFrkYL7QhFkOrbNmFNsTi0U+5Yyu0IRZDq3yF0LUjnUd7od39Tlto
k5AOWuPAXmhT7C5IH/2yHzbjpaYvNSmM81GhjUI6aI1m26aRQvtBSDf5+3ZsXEYP8p96MqMXSul+
HkRn7ueFMnqhjMDPT5lNHQ0zm/QIEuIqG4IWy0hodqHNQpBFYEbU5ze2o0KbhWzQYpUNFiTlEffV
fB61IFP4rhC+yb/QfhASzkeFNsViaDHWjPNRoY1COmi7Y/sgVGizkG6qNQ7shTYK7V1oGndshTYK
6aDdvGMDr6J7PWpBpli89DV2kb7UpCi0ORZDK2xHutGv0H4Q0l2zumPLQb7UQEK6TmtsR4U2xWJo
MdaEA3uhzUI6aLtj+yBEQfsHfcREJ5g/PxwAAAAASUVORK5CYII=
" transform="translate(532, 31)"/>
 
 
 - 
 
 
 3 
 
 
 - 
 
 
 2 
 
 
 - 
 
 
 1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3

## Benchmark results: execution time versus number of gridpoints $N_x$

![alternate text](figs/cputime_vs_size.svg)


**Expectation of $N_x \log N_X$ scaling**. Execution time for this algorithm should ideally be dominated by the $N_x \log N_x$ cost of the FFTs. In the above plot, all the codes do appear to scale as $N_x \log N_x$ at large $N_x$ with different prefactors. 

**Two Julia codes.** 

   * **Julia naive** is a  line-by-line translation of the Matlab code, no attention paid to efficiency
   
   * **Julia** code is nearly a line-by-line translation of the Matlab code, but it eliminates temporary vectors in the inner loop by using in-place FFTs and julia-0.6's loop fusion capability.
  
   * **Julia unrolled** unrolls all the vector operations into explicit for loops. 
  

**Julia, C, C++, Fortran results are practically identical.** The latter two Julia codes, Fortran, C, and C++ beat naive Python and Matlab handily. Julia is close to C and  C++ (factor of 1.06), and Julia unrolled is close to  Fortran (factor of 1.03). Execution times of Julia, Julia unrolled, C, C++, and Fortran are all pretty close, about a 15% spread. The benchmarks were averaged over thousands of runs for $N_x = 32$ scaling down to 8 runs for $N_x = 2^{17}$. 


**CPU, OS, and compiler:** All benchmarks were run single-threaded on a six-core Intel Core i7-3960X CPU @ 3.30GHz with 64 GB memory running openSUSE Leap 42.2. C and C++ were compiled with clang 3.8.0, Fortran with gfortran 4.8.3, Julia was julia-0.7-DEV, and all used optimization ``-O3``. For more details see [benchmark-data/cputime.asc](benchmark-data/cputime.asc). 



## Benchmark results: execution time versus line count

![alternate text](figs/cputime_vs_linecount.svg)

Julia clearly hits the sweet spot of low execution time and low line count. The extra lines in Julia over Matlab are for in-place FFTs. These linecounts exclude whitespace, comments, and plotting. 


# Performance benchmarks, general algorithms

Micro-benchmark codes testing iteration, recursion, matrix operations, and I/O, 
using identical algorithms in 11 languages. Smaller is better. Results normalized so C = 1.


![language benchmarks plot](figs/benchmarks.svg)
